In [111]:
# 8th March 2023 :: @20:44


In [112]:
import numpy as np
import pandas as pd
import torch
import nltk
import re
import spacy
import string
import multiprocessing

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from nltk.corpus import stopwords
from tqdm import tqdm
import tensorflow as tf
import torch
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import matplotlib.pyplot as plt


"""wget.download(
    "https://raw.githubusercontent.com/yogawicaksana/helper_prabowo/main/helper_prabowo_ml.py",
    out="helper_prabowo_ml.py",
)"""

from helper_prabowo_ml import (
    clean_html,
    remove_links,
    remove_special_characters,
    removeStopWords,
    remove_,
    remove_digits,
    lower,
    email_address,
    non_ascii,
    punct,
)


In [113]:
model_name = "jjzha/jobspanbert-base-cased"


In [114]:
# initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


Some weights of the model checkpoint at jjzha/jobspanbert-base-cased were not used when initializing BertModel: ['cls.span_predictions.start_transform.LayerNorm.bias', 'cls.span_predictions.query_start_transform.LayerNorm.weight', 'cls.span_predictions.start_transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.span_predictions.query_end_transform.dense.bias', 'cls.span_predictions.start_classifier', 'cls.span_predictions.query_start_transform.LayerNorm.bias', 'cls.span_predictions.query_end_transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.span_predictions.query_end_transform.LayerNorm.bias', 'cls.span_predictions.query_start_transform.dense.bias', 'cls.span_predictions.end_classifier', 'cls.span_predictions.end_transform.LayerNorm.bias', 'cls.span_predictions.end_transform.dense.weight', 'cls.span_predictions.query_end_transform.LayerNorm.weight', 'cls.span_predictions.end_transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.

In [115]:
nlp = spacy.load("en_core_web_lg")
sp = spacy.load("en_core_web_lg")

nltk.download("stopwords")
nltk.download("punkt")

spacy_st = nlp.Defaults.stop_words
nltk_st = stopwords.words("english")


/home/kalamari/miniconda3/envs/pytorch-gpu/lib/python3.10/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_lg' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.3.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kalamari/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kalamari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [141]:
def jobbert_match(
    resume_df, job_posting_df, model_name=model_name, job_to_resume=False
):

    # preprocess job postings and resumes using BERT tokenizer
    tokenized_job_postings = [
        tokenizer(
            text["title"] + " " + text["description"],
            padding=True,
            truncation=True,
            max_length=512,
        )
        for i, text in job_posting_df.iterrows()
    ]
    tokenized_resumes = [
        tokenizer(text, padding=True, truncation=True, max_length=512)
        for text in resume_df["Resume"]
    ]

    # convert tokenized data to PyTorch tensors
    job_posting_tensors = [
        torch.tensor(text["input_ids"]).unsqueeze(0) for text in tokenized_job_postings
    ]
    resume_tensors = [
        torch.tensor(text["input_ids"]).unsqueeze(0) for text in tokenized_resumes
    ]

    # encode job postings and resumes using BERT model
    job_posting_encodings = [
        model(tensor)[0][:, 0, :].detach().numpy() for tensor in job_posting_tensors
    ]
    resume_encodings = [
        model(tensor)[0][:, 0, :].detach().numpy() for tensor in resume_tensors
    ]

    # reshape the encodings to 2D arrays
    job_posting_encodings = np.vstack(job_posting_encodings)
    resume_encodings = np.vstack(resume_encodings)

    if job_to_resume == True:
        # calculate similarity between job postings and resumes using cosine similarity
        similarity_scores = cosine_similarity(job_posting_encodings, resume_encodings)

        # create a new DataFrame to store the results
        results_df = pd.DataFrame(
            columns=[
                "job_title",
                "description",
                "similarity_score",
                "category",
                "resume",
            ]
        )

        # populate the DataFrame with the raw text and similarity scores
        rows = []
        for i, job_posting in enumerate(job_posting_df["description"]):
            for j, resume in enumerate(resume_df["Resume"]):
                similarity_score = similarity_scores[i][j]
                job_title = job_posting_df.iloc[i]["title"]
                row = {
                    "job_title": job_title,
                    "description": job_posting,
                    "similarity_score": similarity_score,
                    "category": resume_df.iloc[j]["Category"],
                    "resume": resume,
                }
                rows.append(row)

            results_df = pd.concat([results_df, pd.DataFrame(rows)], ignore_index=True)

            # rank the results by descending similarity score
            results_df = results_df.sort_values(by="similarity_score", ascending=False)

            return results_df

    else:
        # calculate similarity between job postings and resumes using cosine similarity
        similarity_scores = cosine_similarity(resume_encodings, job_posting_encodings)

        # create a new DataFrame to store the results
        results_df = pd.DataFrame(
            columns=[
                "category",
                "resume",
                "similarity_score",
                "job_title",
                "description",
            ]
        )

        # populate the DataFrame with the raw text and similarity scores
        rows = []
        for i, resume in enumerate(resume_df["Resume"]):
            for j, job_posting in enumerate(job_posting_df["description"]):
                similarity_score = similarity_scores[i][j]
                job_title = job_posting_df.iloc[j]["title"]
                row = {
                    "category": resume_df.iloc[i]["Category"],
                    "resume": resume,
                    "similarity_score": similarity_score,
                    "job_title": job_title,
                    "description": job_posting,
                }
                rows.append(row)

        results_df = pd.concat([results_df, pd.DataFrame(rows)], ignore_index=True)

        # rank the results by descending similarity score
        results_df = results_df.sort_values(by="similarity_score", ascending=False)

        return results_df


In [142]:
def text_preprocess(data, col):
    data[col] = data[col].apply(func=clean_html)
    data[col] = data[col].apply(func=remove_)
    data[col] = data[col].apply(func=removeStopWords)
    data[col] = data[col].apply(func=remove_digits)
    data[col] = data[col].apply(func=remove_links)
    data[col] = data[col].apply(func=remove_special_characters)
    data[col] = data[col].apply(func=punct)
    data[col] = data[col].apply(func=non_ascii)
    data[col] = data[col].apply(func=email_address)
    data[col] = data[col].apply(func=lower)
    return data


In [143]:
# load job listing and resume datasets
jobs = pd.read_csv("datasets/linkedin-jobs-machine-learning-uk-1936_03082023.csv")
resumes = pd.read_csv("resume_datasets/UpdatedResumeDataSet.csv")


In [144]:
import ast


# Define function to extract key-value pairs from criteria column
def extract_criteria(criteria):
    criteria_dict = {}
    # Convert string to list of dictionaries using ast.literal_eval()
    criteria_list = ast.literal_eval(criteria)
    # Loop through each dictionary in the list
    for d in criteria_list:
        # Loop through each key-value pair in the dictionary
        for key, value in d.items():
            # Add key-value pair to criteria_dict
            criteria_dict[key] = value
    return criteria_dict


# Apply function to 'criteria' column
df_criteria = jobs["criteria"].apply(extract_criteria).apply(pd.Series)

# Concatenate criteria DataFrame with original DataFrame
jobs = pd.concat([jobs.drop("criteria", axis=1), df_criteria], axis=1)

# Display the result
jobs


/tmp/ipykernel_14007/2575025872.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_criteria = jobs["criteria"].apply(extract_criteria).apply(pd.Series)
/tmp/ipykernel_14007/2575025872.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_criteria = jobs["criteria"].apply(extract_criteria).apply(pd.Series)
/tmp/ipykernel_14007/2575025872.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_criteria = jobs["criteria"].apply(extract_criteria).apply(pd.Series)
/tmp/ipykernel_14007/2575025872.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to 

,title,company,description,onsite_remote,salary,location,posted_date,link,Seniority level,Employment type,...,Functie,Bedrijfstakken,Livello di anzianità,Tipo di impiego,Funzione lavorativa,Settori,Nível de experiência,Tipo de emprego,Função,Setores
0,Data Scientist,Airtime Rewards,About UsAirtime Rewards are delivering the lea...,onsite,NaN,"Manchester, England, United Kingdom",2023-03-03,https://uk.linkedin.com/jobs/view/data-scienti...,Not Applicable,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Machine Learning Engineer,orbit,🚀Machine Learning Engineer🎥 Digital Media Serv...,onsite,NaN,"Leeds, England, United Kingdom",2023-02-07,https://uk.linkedin.com/jobs/view/machine-lear...,Not Applicable,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Graduate Machine Learning / Data Scientist,Cambridge Consultants,IntroductionWe are looking for talented mathem...,onsite,NaN,"Cambridge, England, United Kingdom",2023-02-27,https://uk.linkedin.com/jobs/view/graduate-mac...,Entry level,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Data Science Intern,Datasumi,We are looking for a highly motivated and tale...,onsite,NaN,"London, England, United Kingdom",2023-02-20,https://uk.linkedin.com/jobs/view/data-science...,Mid-Senior level,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Machine Learning Engineer (Internship/Part-Time),Terra (YC W21),We are looking for a Machine Learning Engineer...,onsite,NaN,"London, England, United Kingdom",2023-03-01,https://uk.linkedin.com/jobs/view/machine-lear...,Internship,Internship,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,Machine Learning Engineer,Metrica Recruitment,Nan,hybrid,NaN,"Cambridge, England, United Kingdom",2022-12-08,https://uk.linkedin.com/jobs/view/machine-lear...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
765,Data Analyst,Medialab Group,OverviewMedialab’s proprietary marketing intel...,hybrid,NaN,"London, England, United Kingdom",2023-03-03,https://uk.linkedin.com/jobs/view/data-analyst...,Mid-Senior level,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,Graduate/Junior Python Engineer,CareerAddict,Nan,hybrid,NaN,"Wallingford, England, United Kingdom",2023-03-06,https://uk.linkedin.com/jobs/view/graduate-jun...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767,Machine Learning Engineer (StarX),Steel Perlot,StarX is building the financial infrastructure...,hybrid,NaN,"London, England, United Kingdom",2023-03-08,https://uk.linkedin.com/jobs/view/machine-lear...,Mid-Senior level,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
jobs = jobs.rename(
    columns={
        "Seniority level": "seniority_level",
        "Employment type": "employment_type",
        "Job function": "job_function",
        "Industries": "industries",
    }
)


In [146]:
cleaned_jobs = pd.DataFrame(jobs)
cleaned_resumes = pd.DataFrame(resumes)


In [147]:
cleaned_jobs = text_preprocess(cleaned_jobs, "description")
cleaned_resumes = text_preprocess(resumes, "Resume")


In [148]:
# sanity check for job df
cleaned_jobs.head(2)


,title,company,description,onsite_remote,salary,location,posted_date,link,seniority_level,employment_type,...,Functie,Bedrijfstakken,Livello di anzianità,Tipo di impiego,Funzione lavorativa,Settori,Nível de experiência,Tipo de emprego,Função,Setores
0,Data Scientist,Airtime Rewards,about usairtime rewards delivering leading dat...,onsite,NaN,"Manchester, England, United Kingdom",2023-03-03,https://uk.linkedin.com/jobs/view/data-scienti...,Not Applicable,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Machine Learning Engineer,orbit,machine learning engineer digital media servic...,onsite,NaN,"Leeds, England, United Kingdom",2023-02-07,https://uk.linkedin.com/jobs/view/machine-lear...,Not Applicable,Full-time,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
cleaned_resumes.head(2)


,Category,Resume
0,Data Science,skills programming languages python pandas num...
1,Data Science,education details may may b e uitrgpv data sci...


In [150]:
similarity_df2_resume_to_jobs = jobbert_match(
    resume_df=cleaned_resumes[3:4],
    job_posting_df=cleaned_jobs[:10],
    model_name=model_name,
)
similarity_df2_job_to_resumes = jobbert_match(
    resume_df=cleaned_resumes[:10],
    job_posting_df=cleaned_jobs[3:4],
    model_name=model_name,
    job_to_resume=True,
)


In [151]:
similarity_df2_resume_to_jobs


,category,resume,similarity_score,job_title,description
9,Data Science,skills r python sap hana tableau sap hana sql ...,0.946643,Computer Vision Research Engineer - Deep Learning,parkopedia founded mission able answer parking...
3,Data Science,skills r python sap hana tableau sap hana sql ...,0.936520,Data Science Intern,we looking highly motivated talented data scie...
0,Data Science,skills r python sap hana tableau sap hana sql ...,0.936271,Data Scientist,about usairtime rewards delivering leading dat...
5,Data Science,skills r python sap hana tableau sap hana sql ...,0.925238,Data Science Graduate,company descriptionfascinated future captivate...
1,Data Science,skills r python sap hana tableau sap hana sql ...,0.918498,Machine Learning Engineer,machine learning engineer digital media servic...
8,Data Science,skills r python sap hana tableau sap hana sql ...,0.917219,Data Scientist,the opportunityour mission change way world wo...
7,Data Science,skills r python sap hana tableau sap hana sql ...,0.892739,Machine Learning Engineer,machine learning engineer top quant hedge fund...
6,Data Science,skills r python sap hana tableau sap hana sql ...,0.888379,Data Scientist (Graduate Role),we looking data scientist support product sale...
2,Data Science,skills r python sap hana tableau sap hana sql ...,0.869522,Graduate Machine Learning / Data Scientist,introductionwe looking talented mathematicians...
4,Data Science,skills r python sap hana tableau sap hana sql ...,0.832478,Machine Learning Engineer (Internship/Part-Time),we looking machine learning engineer join team...


In [152]:
similarity_df2_job_to_resumes


,job_title,description,similarity_score,category,resume
9,Data Science Intern,we looking highly motivated talented data scie...,0.939823,Data Science,expertise data quantitative analysis decision ...
3,Data Science Intern,we looking highly motivated talented data scie...,0.936520,Data Science,skills r python sap hana tableau sap hana sql ...
7,Data Science Intern,we looking highly motivated talented data scie...,0.935358,Data Science,education details b tech rayat bahra institute...
8,Data Science Intern,we looking highly motivated talented data scie...,0.927704,Data Science,personal skills ability quickly grasp technica...
2,Data Science Intern,we looking highly motivated talented data scie...,0.923133,Data Science,areas interest deep learning control system de...
0,Data Science Intern,we looking highly motivated talented data scie...,0.921417,Data Science,skills programming languages python pandas num...
1,Data Science Intern,we looking highly motivated talented data scie...,0.913756,Data Science,education details may may b e uitrgpv data sci...
6,Data Science Intern,we looking highly motivated talented data scie...,0.907715,Data Science,skills python tableau data visualization r stu...
5,Data Science Intern,we looking highly motivated talented data scie...,0.880009,Data Science,skills c basics iot python matlab data science...
4,Data Science Intern,we looking highly motivated talented data scie...,0.757502,Data Science,education details mca ymcaust faridabad haryan...
